# 統計データと地図データの連携

今回からは、いわゆる主題図の描き方を学びます。
皆さんはいわゆる白地図に別の情報を付け加えて、情報の大小や種類によって色分けして見やすくしたコロプレスマップ(階級区分図)をどこかで目にした事があると思います。
身近なところでは、都道府県毎の新型コロナ感染者数のマップなどは毎日のように報道されています。
コロプレスマップは、領域ごとの違いが一目で把握できる事から、様々な分野で利用されています。
今回はこのコロプレスマップを作成してみたいと思います。
しかし、コロプレスマップに使用するデータを加工するのは意外と手間が掛かります。そこで、APIの仕組みを導入します。整理したデータを、必要な部分だけ持ってこられるのがAPIです。今回はAPIでデータを取得し、そのデータをコロプレスマップの形式で表示させる地図を作ってみたいと思います。

# APIを使って人口データを読み込む

今回は、政府統計のe-Stat(https://www.e-stat.go.jp/)
のAPIを利用します。API機能を使うためには、登録が必要なので、

https://www.e-stat.go.jp/mypage/user/preregister

からユーザー登録をしてください。また、

https://www.e-stat.go.jp/api/api-info/api-guide

に従い、「アプリケーションID」の取得を行う必要もあります。このアプリケーションIDは、のちにPythonコードの中に記述する必要があります。

前回までと同様、Google colaboratoryを使って、Pythonコードを書いて、実行していきます。

まず、必要なライブラリ（これからの作業に必要なプログラムを全部入れた道具箱のようなもの）をインストールします。

*   GDALは、地理情報のラスターとベクターを取り扱うことができるようにするためのライブラリです。

*   geopandasはpandasというデータ解析系に特化したライブラリーの拡張版で、幾何学的（図形や空間）な解析を可能にするライブラリです。

*   matplotlibはグラフ描画のためのライブラリです。

*   foliumはleaflet.jsというインタラクティブな地図の描画を可能にするJava Scriptで書かれたライブラリをPyhton環境に導入するためのライブラリです。

*   pixiedustはデータをGUIのように可視化できるようにするライブラリです。








In [ ]:
!pip install GDAL
!pip install geopandas
!pip install matplotlib
!pip install folium
!pip install pixiedust

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 5.1 MB/s 
     |████████████████████████████████| 16.7 MB 505 kB/s 
     |████████████████████████████████| 6.3 MB 34.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 197 kB 4.9 MB/s 
  Created wheel for pixiedust: filename=pixiedust-1.1.19-py3-none-any.whl size=321803 sha256=3a1b2e6c4934806f3899c5c9c9d9e034bee381548865e8e5b133b1cc81f0f4b2
  Stored in directory: /root/.cache/pip/wheels/05/07/e7/8aca0e820027a63157a916424fd748fb2a2a3e71de5e08eeb8
Successful

次に必要なライブラリを呼び出してきます。

In [ ]:
import numpy as np
import pandas as pd
import urllib.request
import folium
from IPython.display import display

次にe-StatのAPIを使う準備をします。今回はe-Stat APIのバージョン3.0を使用しています。
また、国勢調査のデータを使うことにします。
国勢調査のデータをAPIで指定する番号は、https://www.e-stat.go.jp/statistics/00200521 のように「政府統計コード」の欄に表示があります。この場合、国勢調査の番号は00200521になります。
もし、他の調査データ、例えば、労働力調査であれば、""になります。

http(s)://api.e-stat.go.jp/rest/<バージョン>/app/getStatsList?<パラメータ群>

で　URLを指定するように仕様書に指示があるので、次のように指定して、 統計表情報に関するURLを取得します。

In [ ]:
app_id = "1ae4f04eb1eb24d7afc464c0d0b20698f389200b" #冒頭で取得したアプリケーションID
api_version = "3.0"
base_url = "https://api.e-stat.go.jp/rest/{API_version}/app/".format(API_version=api_version)

get_type = "getStatsList"
stats_code = "00200521" #国勢調査に割り当てられた番号
url = base_url + "{Get_type}?appId={appid}&statsCode={Stats_code}".format(Get_type=get_type,appid=app_id,Stats_code=stats_code)
print(url) # 確認して取得したいデータのIDを調べる

https://api.e-stat.go.jp/rest/3.0/app/getStatsList?appId=1ae4f04eb1eb24d7afc464c0d0b20698f389200b&statsCode=00200521


また、国勢調査の中の個別の調査データを参照したい場合は、stat_data_idの箇所で番号を指定します。
こちらは、ひとつ前で確認した国勢調査全体のリストで調査データ番号を探すこともできますが、

https://www.e-stat.go.jp/dbview?sid=0003445078

のように、直接、e-Stat内を検索して、表示したいデータを検索することもできます。
今回は、令和2年国勢調査

"人口等基本集計　（主な内容：男女・年齢・配偶関係，世帯の構成，住居の状態，母子・父子世帯，国籍など）"の中の、

"男女別人口－全国，都道府県，市区町村（2000年（平成12年）市区町村含む）"データを使用することにしましょう。

e-StatのAPIでは、データ形式はXML, JSON, JSONP, CSVの3つを選ぶことができます(2022年11月現在)。
次のコマンドでは、データ形式はCSVを選択しています。

http(s)://api.e-stat.go.jp/rest/<バージョン>/app/getSimpleStatsData?<パラメータ群>

のように指定すれば、csv形式ですが、getSimpleStatsDataの部分をgetStatsDataと変更すればXML形式、json/getStatsDataとすれば、JSON形式、jsonp/getStatsDataとすればJSONP形式でデータを取得できます。

In [ ]:
get_type="getSimpleStatsData"
stats_data_id="0003445078" # 令和2年の国勢調査ID
url = base_url + "{Get_type}?appId={Appid}&statsDataId={Stats_data_id}".format(Get_type=get_type,Appid=app_id, Stats_data_id=stats_data_id)
print(url) # URLが正しく生成されるか確認する

https://api.e-stat.go.jp/rest/3.0/app/getSimpleStatsData?appId=1ae4f04eb1eb24d7afc464c0d0b20698f389200b&statsDataId=0003445078


リンク先のデータから地図表示に必要なデータのみをピックアップしてきます。

今回は都道府県ごとの人口を地図上に表示させてみましょう。
今回のデータの場合、cd_cat_01は総数の人口を表示するか、男性の人口にするか、女性の人口にするかを指定します。
lv_areaは集計単位で、今回は都道府県ごとの人口を指定します。
section_header_flgは、セクションヘッダーで今回は無しにしておきます。


In [ ]:
get_type="getSimpleStatsData"
stats_data_id="0003445078"
cd_cat_01="0" # 総数0, 男1, 女2
lv_area="2" # 集計レベル。全国レベル　1, 都道府県レベル2, 市区町村レベル　3
section_header_flg="2" # セクションヘッダー無し2
url = base_url + "{Get_type}?appId={Appid}&statsDataId={Stats_data_id}&cdCat01={cdcat01}&lvArea={lv_Area}&sectionHeaderFlg={Section_header_flg}".format(Get_type=get_type,Appid=app_id,Stats_data_id=stats_data_id,cdcat01=cd_cat_01,lv_Area=lv_area, Section_header_flg=section_header_flg)
print(url)

https://api.e-stat.go.jp/rest/3.0/app/getSimpleStatsData?appId=1ae4f04eb1eb24d7afc464c0d0b20698f389200b&statsDataId=0003445078&cdCat01=0&lvArea=2&sectionHeaderFlg=2


取得したURLの中身を読み出します。

In [ ]:
d = urllib.request.urlopen(url).read().decode("utf8")
d

'"tab_code","表章事項","cat01_code","男女","area_code","全国，都道府県，市区町村（2000年市区町村含む）","time_code","時間軸（年次）","unit","value","annotation"\n"2020_01","人口","0","総数","01000","北海道","2020000000","2020年","人","5224614",""\n"2020_01","人口","0","総数","02000","青森県","2020000000","2020年","人","1237984",""\n"2020_01","人口","0","総数","03000","岩手県","2020000000","2020年","人","1210534",""\n"2020_01","人口","0","総数","04000","宮城県","2020000000","2020年","人","2301996",""\n"2020_01","人口","0","総数","05000","秋田県","2020000000","2020年","人","959502",""\n"2020_01","人口","0","総数","06000","山形県","2020000000","2020年","人","1068027",""\n"2020_01","人口","0","総数","07000","福島県","2020000000","2020年","人","1833152",""\n"2020_01","人口","0","総数","08000","茨城県","2020000000","2020年","人","2867009",""\n"2020_01","人口","0","総数","09000","栃木県","2020000000","2020年","人","1933146",""\n"2020_01","人口","0","総数","10000","群馬県","2020000000","2020年","人","1939110",""\n"2020_01","人口","0","総数","11000","埼玉県","2020000000","2020年","人","7344765",""\n"2020_01","人口","0","総数","1

In [ ]:
dlines = d.splitlines()[1:] #改行で分割
dlines

['"2020_01","人口","0","総数","01000","北海道","2020000000","2020年","人","5224614",""',
 '"2020_01","人口","0","総数","02000","青森県","2020000000","2020年","人","1237984",""',
 '"2020_01","人口","0","総数","03000","岩手県","2020000000","2020年","人","1210534",""',
 '"2020_01","人口","0","総数","04000","宮城県","2020000000","2020年","人","2301996",""',
 '"2020_01","人口","0","総数","05000","秋田県","2020000000","2020年","人","959502",""',
 '"2020_01","人口","0","総数","06000","山形県","2020000000","2020年","人","1068027",""',
 '"2020_01","人口","0","総数","07000","福島県","2020000000","2020年","人","1833152",""',
 '"2020_01","人口","0","総数","08000","茨城県","2020000000","2020年","人","2867009",""',
 '"2020_01","人口","0","総数","09000","栃木県","2020000000","2020年","人","1933146",""',
 '"2020_01","人口","0","総数","10000","群馬県","2020000000","2020年","人","1939110",""',
 '"2020_01","人口","0","総数","11000","埼玉県","2020000000","2020年","人","7344765",""',
 '"2020_01","人口","0","総数","12000","千葉県","2020000000","2020年","人","6284480",""',
 '"2020_01","人口","0","総数","13000","東京都","

In [ ]:
pcodes = []
names = []
populations = []
dlines

['"2020_01","人口","0","総数","01000","北海道","2020000000","2020年","人","5224614",""',
 '"2020_01","人口","0","総数","02000","青森県","2020000000","2020年","人","1237984",""',
 '"2020_01","人口","0","総数","03000","岩手県","2020000000","2020年","人","1210534",""',
 '"2020_01","人口","0","総数","04000","宮城県","2020000000","2020年","人","2301996",""',
 '"2020_01","人口","0","総数","05000","秋田県","2020000000","2020年","人","959502",""',
 '"2020_01","人口","0","総数","06000","山形県","2020000000","2020年","人","1068027",""',
 '"2020_01","人口","0","総数","07000","福島県","2020000000","2020年","人","1833152",""',
 '"2020_01","人口","0","総数","08000","茨城県","2020000000","2020年","人","2867009",""',
 '"2020_01","人口","0","総数","09000","栃木県","2020000000","2020年","人","1933146",""',
 '"2020_01","人口","0","総数","10000","群馬県","2020000000","2020年","人","1939110",""',
 '"2020_01","人口","0","総数","11000","埼玉県","2020000000","2020年","人","7344765",""',
 '"2020_01","人口","0","総数","12000","千葉県","2020000000","2020年","人","6284480",""',
 '"2020_01","人口","0","総数","13000","東京都","

データを加工しやすい形に成形します。最終的に引数dfにデータを渡します。

In [ ]:
for line in dlines:
    line2 = line.replace('"', "").split(",")
    pcode = line2[4]
    name = line2[5]
    population = line2[9]
    pcode = pcode[0:2]
    population = int(population)
    pcodes.append(pcode)
    names.append(name)
    populations.append(population)

df = pd.DataFrame({"pcode" : pcodes, "name" : names, "population" : populations})
display(df)

,pcode,name,population
0,01,北海道,5224614
1,02,青森県,1237984
2,03,岩手県,1210534
3,04,宮城県,2301996
4,05,秋田県,959502
5,06,山形県,1068027
6,07,福島県,1833152
7,08,茨城県,2867009
8,09,栃木県,1933146
9,10,群馬県,1939110


# 地図の描画

次に、成形したデータ(df)を地図上に示します。そのためには、まず、ベースとなる地図を読み込みます。ここでは、foliumというパッケージを使って、leaflet.jsというインタラクティブな地図の描画を行いましょう。
なお、デフォルトで、背景地図には、
”OpenStreetMap” ”Stamen Terrain”, “Stamen Toner”, “Stamen Watercolor” ”CartoDB positron”, “CartoDB dark_matter” が使えます。
最終的にmapという引数に背景地図の情報を渡しておきます。

In [ ]:
location = [32.99125000,138.45999999] #地図の中心位置を指定。
tiles='CartoDB positron' #背景地図の指定。
zoom_start = 5 #ズームレベル
map = folium.Map(location=location, tiles=tiles, zoom_start=zoom_start)
map

背景地図が表示できたら、次に必要なのは、人口データを表示させるための枠になる都道府県ごとに境界が示されたベクタデータです。今回はgeojsonという形式のファイルで読み込んでいきます。
今回はあらかじめ、
https://github.com/wata909/interface2022/raw/main/GIS_DATA/japan.geojson

の場所に都道府県境界のあるベクタデータのgeojsonファイルを置いておきました。それをcolaboratory環境に読み込みます。もし、皆さんが各自で似たような地図を作りたい場合は、コンピュータ内やクラウド上にgeojsonファイルを置き、それを読み込むことで同じように地図データを取り込むことが可能です。

In [ ]:
!pip install geojson
import geojson
import geopandas as gpd
jpn = "https://github.com/wata909/interface2022/raw/main/GIS_DATA/japan.geojson"
fjpn = gpd.read_file(jpn) #ベクタファイルの読み込み
fjpn #読み込んだデータの確認

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


,coc,nam,CODE,pref_code,geometry
0,JPN,AICHI,23,23,"MULTIPOLYGON Z (((136.96091 35.41453 0.00000, ..."
1,JPN,AKITA,5,05,"MULTIPOLYGON Z (((140.87970 40.51147 0.00000, ..."
2,JPN,AOMORI,2,02,"MULTIPOLYGON Z (((140.91240 41.55253 0.00000, ..."
3,JPN,CHIBA,12,12,"MULTIPOLYGON Z (((139.77620 36.09087 0.00000, ..."
4,JPN,EHIME,38,38,"MULTIPOLYGON Z (((132.99680 34.28980 0.00000, ..."
5,JPN,FUKUI,18,18,"MULTIPOLYGON Z (((136.24370 36.29560 0.00000, ..."
6,JPN,FUKUOKA,40,40,"MULTIPOLYGON Z (((130.10440 34.23940 0.00000, ..."
7,JPN,FUKUSHIMA,7,07,"MULTIPOLYGON Z (((139.68111 37.84720 0.00000, ..."
8,JPN,GIFU,21,21,"MULTIPOLYGON Z (((137.23911 36.45180 0.00000, ..."
9,JPN,GUNMA,10,10,"MULTIPOLYGON Z (((139.09700 37.05880 0.00000, ..."


無事、geojsonファイルが読み込めて、そこに都道府県ごとの位置情報が付加されていることが確認できれば、成功です。
これでいよいよ人口規模に従って、都道府県を色分けする作業に取り掛かれます。
folium.Choropleth関数を使い、パラメータを指定していくことで、思った地図を描くことができます。

先ほど作った背景地図"map"に色分けの情報を追加するという形で、コロプレスマップは作成することができます。
また、表示に使用しているleafletはインタラクティブな地図表示なので、左上の＋ーボタンで地図の拡大縮小が可能ですし、地図上でドラッグ&ドロップ操作をすることで、地図表示の位置を自由に変えることが可能です。

In [ ]:
# 地図に階級ごとに色を塗る
folium.Choropleth(
geo_data=jpn, #都道府県ごとの緯度経度情報のあるgeojsonデータ
name='choropleth',
data=df, #描画データ
columns=["pcode", "population"], # ["都道府県コード", "値の列"]
key_on='properties.pref_code',
threshold_scale=[500000, 1000000, 3000000, 10000000, 15000000],
fill_color='YlGnBu',  #色分けするための色指定
fill_opacity=0.7,  #塗りつぶしの濃さ
line_opacity=0.2, #境界ラインの濃さ
).add_to(map)

display(map)

NameError: ignored

以上、APIの統計データを使って、それを地図上に読み込み、表示させる手順と、その表示にはコロプレスマップを使用する方法を解説しました。次回は、複数の統計データを同時に地図上に表示する方法について学んでいきます。